In [3]:
# !pip install transformers
# !pip install datasets
# !pip install scipy transformers
!pip install torch
!pip install pandas
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
# import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, ClassLabel
import pandas as pd

csv_file = "left_and_center.csv"
right_csv_file = "right.csv"
df_left_center = pd.read_csv(csv_file)
df_right = pd.read_csv(right_csv_file)

# Concatenate the datasets
df = pd.concat([df_left_center, df_right])

# Ensure labels are integers and map them to ClassLabel
label_mapping = {'left': 0, 'center': 1, 'right': 2}  # Adjust according to your labels
df['label'] = df['Leaning'].map(label_mapping)

sampled_df = df.groupby('label').apply(lambda x: x.sample(n=97, random_state=42)).reset_index(drop=True)

# Create the dataset and convert label column to ClassLabel type
dataset = Dataset.from_pandas(sampled_df.rename(columns={'Text': 'text', 'label': 'label'}))
dataset = dataset.cast_column('label', ClassLabel(names=['left', 'center', 'right']))


<ipython-input-8-d65052441f14>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('label').apply(lambda x: x.sample(n=97, random_state=42)).reset_index(drop=True)


Casting the dataset:   0%|          | 0/291 [00:00<?, ? examples/s]

In [6]:
print(dataset)

Dataset({
    features: ['text', 'Leaning', 'Year', 'label'],
    num_rows: 291
})


In [10]:
#!conda create -n bert_env python=3.9
#!conda activate bert_env
#!pip install "numpy<2" scipy transformers datasets pandas
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [11]:
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Split the dataset into train and eval sets
dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="label")

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Tokenize datasets with streaming and caching
train_dataset = dataset["train"].map(
    tokenize_function,
    batched=True,
    load_from_cache_file=True,
    batch_size=1000,  # Process in small chunks
)

eval_dataset = dataset["test"].map(
    tokenize_function,
    batched=True,
    load_from_cache_file=True,
    batch_size=1000,  # Process in small chunks
)

# Set the format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Initialize the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Load the metric (accuracy)
accuracy_metric = evaluate.load("accuracy")

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)  # Get predicted class
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    learning_rate=2e-5,
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,  # Reduce batch size to save memory
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",
    report_to=[],  # Disable WandB
    fp16=True,  # Use mixed precision for memory efficiency
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add accuracy computation
)

# Train the model
trainer.train()

# Evaluate the model on the evaluation set
eval_results = trainer.evaluate()

# Print evaluation metrics
print(f"Evaluation Results: {eval_results}")


Map:   0%|          | 0/232 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-5d214a89f77e>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.093600,1.127205,0.237288
2,1.078300,1.096380,0.305085


Evaluation Results: {'eval_loss': 1.0963804721832275, 'eval_accuracy': 0.3050847457627119, 'eval_runtime': 95.5379, 'eval_samples_per_second': 0.618, 'eval_steps_per_second': 0.084, 'epoch': 2.0}


In [12]:
trainer.evaluate()

{'eval_loss': 1.0963804721832275,
 'eval_accuracy': 0.3050847457627119,
 'eval_runtime': 96.0192,
 'eval_samples_per_second': 0.614,
 'eval_steps_per_second': 0.083,
 'epoch': 2.0}

In [13]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')